In [44]:
#!pip install hyperopt

# Library Import

In [23]:
# Basic Library
import numpy as np
import pandas as pd
import time
import psutil
import warnings
warnings.filterwarnings("ignore")

# Visualization Library
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc("font", family="Apple Gothic") # 제출할 때 이거 지우고..
#plt.rc("font", family="Malgun Gothic") # 제출할 때 이걸로!!
plt.rc("axes", unicode_minus = False)
%matplotlib inline

# Data Preprocessing (feature engineering)
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Modeling
from sklearn import datasets, model_selection
#from sklearn import ensemble
#from sklearn import tree
# Tree 계열 Library
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
# SVC Library
from sklearn.svm import SVC
# Boosting 계열 Library
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, plot_importance
import xgboost as xgb
# Catboost 쪽 Library
from sklearn.cluster import KMeans # ??? 이거 뭐지?
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
# LightGBM
#from lightgbm import LGBMRegressor, LGBMClassifier
#from vecstack import stacking

# Hyperparameter tuning Library
from sklearn.model_selection import GridSearchCV
import optuna
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Metrics
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc

## Data Loading

In [24]:
df = pd.read_csv('Documents/data/preprocessing.csv')

In [25]:
df = df.drop(['Unnamed: 0'],axis=1)
df_x = df.drop(['credit'],axis=1)
df_y = df[['credit']]

In [26]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_x, df_y, test_size=0.3, random_state=0)

In [27]:
# preprocessing
# feature classification (categorical vs numerical)

categorical_features = ['gender', 'car', 'reality', 'income_type',
                        'edu_type', 'family_type', 'house_type',
                        'work_phone', 'phone','email', 'occyp_type']
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore')

numerical_features = ['income_total', 'begin_month', 'Age', 'worked_year']
numerical_transformer = MinMaxScaler()

pass_through = ['child_num', 'family_size']

pass

preprocessor = ColumnTransformer(
    transformers=[('cat', categorical_transformer, categorical_features),
                  ('num', numerical_transformer, numerical_features),
                  ('passthrough', 'passthrough', pass_through)])

preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)])
preprocessor_pipe.fit(train_x)

train_x_transformed = preprocessor_pipe.transform(train_x)
test_x_transformed = preprocessor_pipe.transform(test_x)

In [28]:
train_x_transformed.shape

(18519, 58)

In [29]:
test_x_transformed.shape

(7938, 58)

In [30]:
train_y.shape

(18519, 1)

In [31]:
test_y.shape

(7938, 1)

# ML Model

- Tree : Random Forest
- SVM : Support Vector Classifier (with Kernel)
- Boosting : XG Boosting
- Boosting : Cat Boosting
- Model staking : (using vecstack)
- Model stacking : (using soft voting)


In [32]:
# 성능 평가를 위한 memory_usage 함수 생성
def memory_usage(message: str = 'debug'):
    # current process RAM usage
    p = psutil.Process()
    rss = p.memory_info().rss / 2 ** 20 # Bytes to MB
    print(f"[{message}] memory usage: {rss: 10.5f} MB")

## 1. Random Forest

In [39]:
def RF_clf():
    train_start = time.time()
    # RF Modeling (using GridSearchCV)
    params = {'n_estimators': [10, 200], 'max_depth' : [6, 8, 10, 12],
              'min_samples_split' : [2, 4, 6, 8, 10], 'min_samples_leaf' : [2, 4, 6, 8, 10]}
    RF_clf = RandomForestClassifier(random_state=0, n_jobs= -1)
    grids_cv = GridSearchCV(RF_clf, param_grid=params, cv =3, n_jobs=-1)
    grids_cv.fit(train_x_transformed,train_y)

    print('The best parameters are ', grids_cv.best_params_)
    print('best: {:.4f}'.format(grids_cv.best_score_))
    memory_usage("학습하는데 걸린 시간  {:.2f} 분\n".format( (time.time() - train_start)/60))

In [42]:
RF_clf()

/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

The best parameters are  {'max_depth': 12, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
best: 0.6927
[학습하는데 걸린 시간  1.67 분
] memory usage:  101.62500 MB


/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dklee/miniforge3/envs/jedi/lib/python3.8/site-packages/sklearn/mod

In [43]:
RF_clf_best = RandomForestClassifier(grids_cv.best_params_)
RF_clf_best.fit(train_x,train_y)
pred_RF_best = RF_clf_best.predict(test_x)
print('accuracy : {:.4f}'.format(accuracy_score(test_y,pred_RF_best)))

# feature importance
ftr_import_val = RF_clf1.feature_importances_
ftr_import = pd.Series(ftr_import_val, index = train_x.columns)
ftr_top20 = ftr_import.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Top 20 Feature Importances')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()

NameError: name 'grids_cv' is not defined

In [20]:
# # best parameter로 다시 학습 (transformed된 data로) -> transformed된걸로 어떻게 시각화 못 하나?
# RF_clf2 = RandomForestClassifier(n_estimators=200,max_depth=12,min_samples_leaf=2,
#                                  min_samples_split=2,random_state=0,n_jobs=-1)
# RF_clf2.fit(train_x_transformed,train_y)
# pred_RF2 = RF_clf2.predict(test_x_transformed)
# print('accuracy : {:.4f}'.format(accuracy_score(test_y,pred_RF2)))

## 2. SVM

In [21]:
def svm_clf():
    # train 시작 시간
    train_start = time.time()
    reg_candidate = np.arange(0.005,1.0,0.01)
    space={'n_estimators': hp.quniform('n_estimators', 1000, 3000, 300),
           'C' : hp.quniform('C', 0.005, 1, 0.01),
           'gamma' : hp.quniform('gamma', 0.005, 1, 0.01)}
    
    def hyperparameter_tuning(space):
        model=SVC(C = space['C'], gamma = space['gamma'], random_state=42)
        evaluation = [(train_x_transformed, train_y), (test_x_transformed, test_y)]
        model.fit(train_x_transformed, train_y)    
        accuracy = accuracy_score(model.predict(test_x_transformed), test_y)
        # 평가 방식 선정
        return {'loss': 1 - accuracy, 'status': STATUS_OK, 'model': model}# Trials 객체 선언합니다.

    trials = Trials()
    # best에 최적의 하이퍼 파라미터를 return 받습니다.
    best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=20, # 최대 반복 횟수를 지정합니다.
            trials=trials)
    # 최적화된 결과를 int로 변환해야하는 파라미터는 타입 변환을 수행합니다.
    print (best) # {'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 1400, 'reg_alpha': 0.001, 'reg_lambda': 1e-05, 'subsample': 0.7000000000000001}
    print(trials.best_trial['result']) #{'C': 0.9400000000000001, 'gamma': 0.27} [40:04<00:00, 120.23s/trial, best loss: 0.328193499622071]
    memory_usage("학습하는데 걸린 시간  {:.2f} 분\n".format( (time.time() - train_start)/60))

In [22]:
svm_clf()

100%|█████████| 20/20 [11:06<00:00, 33.35s/trial, best loss: 0.3347190728143109]
{'C': 0.87, 'gamma': 0.86, 'n_estimators': 1200.0}
{'loss': 0.3347190728143109, 'status': 'ok', 'model': SVC(C=0.87, gamma=0.86, random_state=42)}
[학습하는데 걸린 시간  11.12 분
] memory usage:  722.90625 MB


## 3. xgboost

In [15]:
def objective(trial):
    params = {
        "objective": "multi:softprob",
        "eval_metric":'mlogloss',
        "booster": 'gbtree',
        #'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'gpu_id': 0, # GPU 사용시
        "tree_method": 'exact', 'gpu_id': -1,  # CPU 사용시
        "verbosity": 0,
        'num_class':3,
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        'n_estimators': trial.suggest_int("n_estimators", 1000, 10000, step=100),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-2, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-2, 1),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),     
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        "gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True),
        # 'num_parallel_tree': trial.suggest_int("num_parallel_tree", 1, 500) 추가하면 느려짐.
    }

    model = xgb.XGBClassifier(**params, random_state = 42, use_label_encoder = False)

    bst = model.fit(train_x_transformed, train_y,eval_set=[(test_x_transformed,test_y)], early_stopping_rounds=50, verbose=False)
    preds = bst.predict(test_x_transformed)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
    return accuracy

In [ ]:
train_start = time.time()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")

trial = study.best_trial

print("  Accuracy: {}".format(trial.value))
print("  Best hyperparameters: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    
xgb_model = xgb.XGBClassifier(**study.best_params, random_state = 42, use_label_encoder = False)
xgb_model.fit(train_x_transformed, train_y)

accuracy = accuracy_score(model.predict(test_x_transformed), y_test)

print("Accuracy: {}".format(accuracy))
memory_usage("학습하는데 걸린 시간  {:.2f} 분\n".format( (time.time() - train_start)/60))

## 4. catboost

In [ ]:
train = pd.read_csv('Documents/data/preprocessing.csv')

In [ ]:
for df in [train]:
    df['income_total'] = np.log1p(1+df['income_total'])

In [ ]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

In [ ]:
n_est = 2000
seed = 42
n_fold = 8
n_class = 3

target = 'credit'
X = train.drop(target, axis=1)
y = train[target]
X_test = train


skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class)) # np.zeros(shape, dtype, ordaer) : 0으로 가득찬 array 생성
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)  # catboost lib 에서 제공하는 Pool()을 이용해서 편하게 묶음
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier()
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
  cat_pred_test += model_cat.predict_proba(X_test) / n_fold
  print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')

# Model metrics

In [ ]:
# model metric 

def modeling_metrics(test_y, pred, pred_proba):
    confusion = confusion_matrix(test_y, pred)
    accuracy = accuracy_score(test_y, pred)
    precision = precision_score(test_y, pred)
    recall = recall_score(test_y, pred)
    f1 = f1_score(test_y, pred)
    roc_auc = roc_auc_score(test_y, pred_proba)
    return [accuracy, precision, recall, f1, roc_auc]

In [ ]:
#ROC_CURVE 코드 작성 후, 그래프 도출.